# Extracting results from the template matching search

After running through a template matching process, we have a set of location, correlation, and orientation statistics that need parsed. For example, peaks need to be picked from the z-score map, and the corresponding best orientations and defoci at those locations need to be read.

While the necessary information *can* be grabbed from a configuration file, having a tabular data format is more flexible to work with.

In [ ]:
"""Extracting results to DataFrame."""

from leopard_em.pydantic_models.managers import MatchTemplateManager

### Importing a configuration

Here, we use an example configuration file to demonstrate how to extract the relevant information.
If you've already run template matching, then this information should already be present in the `MatchTemplateManager` instance.

In [ ]:
yaml_path = "/some/path/to/your/yaml_file.yaml"
mt_manager = MatchTemplateManager.from_yaml(yaml_path)
mt_result = mt_manager.match_template_result
mt_result.load_tensors_from_paths()  # Needed to load results into memory


# Manually set the number of correlations. Automatically calculated after a run
total_corr = int(13 * 1.59e6)
mt_result.total_projections = total_corr

## Extracting minimal peak information

Minimal information at the peaks (location, orientation, peak height) can be extracted from the `MatchTemplateResult` instance as either a `dict` or `pandas.DataFrame`. Here, we show how to extract the information as a `pandas.DataFrame`.

In [ ]:
mt_result.locate_peaks()
df_peaks = mt_result.peaks_to_dataframe()
df_peaks

## Converting full result information to a DataFrame

The `MatchTemplateManager` class also has a method -- `results_to_dataframe` -- which populates a `pandas.DataFrame` with all necessary information for (most) downstream processing.

In [ ]:
df_full = mt_manager.results_to_dataframe()
df_full